In [82]:
from genlm.eval.domains.pattern_matching import (
    PatternMatchingDataset,
    PatternMatchingEvaluator,
)
dataset = PatternMatchingDataset.from_csv("/Users/yemara/Desktop/genlm/genlm-eval/assets/pattern_matching/patterns.csv", pattern_column="regex")
evaluator = PatternMatchingEvaluator()

In [83]:
from genlm.control import PromptedLLM, AWRS
from genlm.eval import ModelOutput, ModelResponse
from genlm.eval.domains.pattern_matching import (
    default_prompt_formatter,
    PatternPotential,
)

# Load an LLM
LLM = PromptedLLM.from_name("gpt2", eos_tokens=[b"\n", b"\n\n"])


async def model(instance, output_dir, replicate):
    # Set the prompt for the LLM.
    LLM.prompt_ids = default_prompt_formatter(
        LLM.model.tokenizer, instance, use_chat_format=False
    )
    print(LLM.prompt_ids)

    # Define a potential that ensures the generated text matches the pattern
    potential = PatternPotential(instance.pattern).coerce(LLM, f=b"".join)

    # Define an adaptive weighted rejection sampler to sample tokens from the constrained model.
    sampler = AWRS(LLM, potential)

    # Run SMC to sample sequences from the constrained model.
    sequences = await sampler.smc(
        n_particles=1,
        ess_threshold=0.5,
        max_tokens=100,
        verbosity=1,
    )

    # Return the sampled sequences and their probabilities as a ModelOutput.
    return ModelOutput(
        responses=[
            ModelResponse(response=sequence, weight=prob)
            for sequence, prob in sequences.decoded_posterior.items()
        ],
    )

In [84]:
from genlm.eval import run_evaluation

results = await run_evaluation(
    dataset=dataset,
    model=model,
    evaluator=evaluator,
    max_instances=20,
    n_replicates=1,
    verbosity=1,
    # output_dir="pattern_matching_results", optionally save the results to a directory
)

[1639, 389, 257, 7613, 8796, 326, 18616, 13042, 12336, 3218, 14700, 13, 5514, 5072, 262, 2748, 4731, 326, 7466, 262, 40364, 3912, 11, 2147, 517, 13, 198, 47546, 25, 357, 397, 47762, 198, 26410, 25, 450, 198, 47546, 25, 357, 397, 91, 10210, 47762, 198, 26410, 25, 22927, 198, 47546, 25, 685, 64, 12, 89, 48688, 198, 26410, 25, 23748, 198, 7, 30, 27, 0, 59, 67, 90, 18, 30072, 39305, 7, 12248, 59, 67, 90, 18, 30072]
-3.82:	[a]
-12.28:	[a|b]
-17.86:	[a|b|c]
-20.47:	[a|b|c|EOS]
Instance instance_id=0 pattern='(?<!\\d{3})abc(?!\\d{3})'
Mean weighted accuracy (instance): 1.0
Mean weighted accuracy (total): 1.0

[1639, 389, 257, 7613, 8796, 326, 18616, 13042, 12336, 3218, 14700, 13, 5514, 5072, 262, 2748, 4731, 326, 7466, 262, 40364, 3912, 11, 2147, 517, 13, 198, 47546, 25, 357, 397, 47762, 198, 26410, 25, 450, 198, 47546, 25, 357, 397, 91, 10210, 47762, 198, 26410, 25, 22927, 198, 47546, 25, 685, 64, 12, 89, 48688, 198, 26410, 25, 23748, 198, 61, 7, 30, 91, 7, 64, 14726, 7, 65, 14726, 7, 66, 40

# Spider (text2sql)

In [86]:
from genlm.eval.domains.spider import SpiderDataset, SpiderEvaluator

In [87]:
spider_data_dir = "/Users/yemara/Desktop/genlm/genlm-eval/assets/spider/spider_sample"  # Replace with your path to the spider dataset
spider_grammars = "/Users/yemara/Desktop/genlm/genlm-eval/assets/spider/grammars.json"  # Replace with your path to the spider grammars

dataset = SpiderDataset.from_spider_dir(
    spider_data_dir, grammar_json_path=spider_grammars, few_shot_example_ids=[0, 1]
)

evaluator = SpiderEvaluator(spider_data_dir)

In [ ]:
from genlm.control import PromptedLLM, AWRS, BoolCFG
from genlm.eval import ModelOutput, ModelResponse
from genlm.eval.domains.spider import default_prompt_formatter

# Load an LLM
LLM = PromptedLLM.from_name("gpt2", eos_tokens=[b"\n", b"\n\n"])


async def model(instance, output_dir, replicate):
    # Set the prompt for the LLM.
    LLM.prompt_ids = default_prompt_formatter(
        LLM.model.tokenizer, instance, use_chat_format=False
    )

    # Define a potential that ensures the generated text matches the pattern
    potential = BoolCFG.from_lark(instance.lark_grammar).coerce(LLM, f=b"".join)

    # Define an adaptive weighted rejection sampler to sample tokens from the constrained model.
    sampler = AWRS(LLM, potential)

    # Run SMC to sample sequences from the constrained model.
    sequences = await sampler.smc(
        n_particles=1,
        ess_threshold=0.5,
        max_tokens=100,
        verbosity=1,
    )

    # Return the sampled sequences and their probabilities as a ModelOutput.
    return ModelOutput(
        responses=[
            ModelResponse(response=sequence, weight=prob)
            for sequence, prob in sequences.decoded_posterior.items()
        ],
    )

In [93]:
from genlm.eval import run_evaluation

results = await run_evaluation(
    dataset=dataset,
    model=model,
    evaluator=evaluator,
    max_instances=10,
    n_replicates=1,
    verbosity=1,
    # output_dir="spider_results", optionally save the results to a directory
)

-0.00:	[␣SELECT]
-0.00:	[␣SELECT]
-0.00:	[␣SELECT|␣count]
-0.00:	[␣SELECT|␣1]
-0.00:	[␣SELECT|␣count|(]
-0.00:	[␣SELECT|␣1|␣FROM]
-0.00:	[␣SELECT|␣count|(|*)]
-0.01:	[␣SELECT|␣1|␣FROM|␣concert]
-0.00:	[␣SELECT|␣count|(|*)|␣FROM]
-0.01:	[␣SELECT|␣1|␣FROM|␣concert|EOS]
-0.00:	[␣SELECT|␣count|(|*)|␣FROM|␣singer]
-0.00:	[␣SELECT|␣count|(|*)|␣FROM|␣singer|EOS]
Instance utterance: How many singers do we have?, schema_name: concert_singer (id: 0)
Mean weighted accuracy (instance): 0.5003311325295734
Mean weighted accuracy (total): 0.5003311325295734

-0.00:	[␣SELECT]
-0.00:	[␣SELECT]
-0.00:	[␣SELECT|␣d]
-0.47:	[␣SELECT|␣count]
-6.55:	[␣SELECT|␣d|IST]
-0.47:	[␣SELECT|␣count|(]
-7.70:	[␣SELECT|␣d|IST|IN]
-0.47:	[␣SELECT|␣count|(|*)]
-7.70:	[␣SELECT|␣d|IST|IN|CT]
-0.47:	[␣SELECT|␣count|(|*)|␣FROM]
-10.23:	[␣SELECT|␣d|IST|IN|CT|␣(]
-0.47:	[␣SELECT|␣count|(|*)|␣FROM|␣singer]
-12.59:	[␣SELECT|␣d|IST|IN|CT|␣(|*)]
-0.47:	[␣SELECT|␣count|(|*)|␣FROM|␣singer|EOS]
-12.59:	[␣SELECT|␣d|IST|IN|CT|␣(|*)|␣FRO

In [75]:
import pandas as pd
path = "/Users/yemara/Desktop/genlm/genlm-eval/assets/pattern_matching/patterns.csv"
print(pd.read_csv(path, nrows=0).columns.tolist())  # see the header names


['regex']


In [76]:
dataset = PatternMatchingDataset.from_csv("/Users/yemara/Desktop/genlm/genlm-eval/assets/pattern_matching/patterns.csv", pattern_column="regex")
instance = next(iter(dataset))

In [68]:
from itertools import islice
first_k = list(islice(dataset, 5))
first_k

[pattern: (?<!\d{3})abc(?!\d{3}) (id: 0),
 pattern: ^(?|(a)|(b)|(c))\1$ (id: 1),
 pattern: [\p{IsAlphabetic}&&[\P{L}]] (id: 2),
 pattern: ^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$ (id: 3),
 pattern: ^[a-f0-9]{8}-[a-f0-9]{4}-[1-5][a-f0-9]{3}-[89ab][a-f0-9]{3}-[a-f0-9]{12}$ (id: 4)]

In [77]:
prompt_ids = default_prompt_formatter(bytelm.llm.tokenizer, instance, use_chat_format=False)
bytelm.set_prompt_from_str(bytelm.llm.tokenizer.decode(prompt_ids))

In [78]:
potential = PatternPotential(instance.pattern).coerce(bytelm, f=b"".join)

In [79]:
sampler = AWRS(bytelm, potential)

In [80]:
sequences = await sampler.smc(
        n_particles=5,
        ess_threshold=0.5,
        max_tokens=100,
        verbosity=1,
    )

Task was destroyed but it is pending!
task: <Task cancelling name='Task-74548' coro=<AsyncTokenByteTrie._background_loop() running at /Users/yemara/miniconda3/envs/control/lib/python3.12/site-packages/genlm/bytes/trie.py:570> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-74549' coro=<AsyncTokenByteTrie._background_loop() running at /Users/yemara/miniconda3/envs/control/lib/python3.12/site-packages/genlm/bytes/trie.py:570> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-74550' coro=<AsyncTokenByteTrie._background_loop() running at /Users/yemara/miniconda3/envs/control/lib/python3.12/site-packages/genlm/bytes/trie.py:570> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-74551' coro=<AsyncTokenByteTrie._background_loop() running at /Users/yemara/miniconda3/envs/control/lib/python3.12/site-packages/genlm/bytes/trie.py:570> wait_for=

AssertionError: Beam is empty

In [43]:
instance

pattern: (?<!\d{3})abc(?!\d{3}) (id: 0)

In [45]:
from genlm.control import BoolFSA
a = "?(<!\d{3})abc(?!\d{3})"
fsa = BoolFSA.from_regex(a)
bytelm = ByteLLM.from_name(
    "gpt2",
    beam_params=BeamParams(K=10, prune_threshold=0.0, eos_tokens={b"\n", b"\n\n"})
)
sampler = AWRS(bytelm, fsa.coerce(bytelm, f=b"".join))

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/var/folders/q6/dlj0nzss0_scbhn5lq54pplr0000gn/T/ipykernel_6899/973974669.py:2: SyntaxWarning: invalid escape sequence '\d'
  a = "?(<!\d{3})abc(?!\d{3})"
/var/folders/q6/dlj0nzss0_scbhn5lq54pplr0000gn/T/ipykernel_6899/973974669.py:2: SyntaxWarning: invalid escape sequence '\d'
  a = "?(<!\d{3})abc(?!\d{3})"


InvalidSyntax: 